# [사전작업] 챗봇 기본구조 확인 (Streamlit)

In [ ]:
# 사전 정의된 기본 챗봇 애플리케이션 
!cp ./demo-app.py ./basic.py ..

# [Lab1] 지식 문서 업로드 기능 구현 

In [ ]:
!pip install -r requirements.txt -q

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import re
import json
import pdfplumber
from datetime import datetime
from langchain_core.documents import Document

In [ ]:
import boto3
from utils.ssm import parameter_store

region=boto3.Session().region_name
pm = parameter_store(region)

opensearch_domain_endpoint = pm.get_params(key="opensearch_domain_endpoint", enc=False)
opensearch_user_id = pm.get_params(key="opensearch_user_id", enc=False)
opensearch_user_password = pm.get_params(key="opensearch_user_password", enc=True)

In [ ]:
opensearch_domain_endpoint

### PDF 문서 처리방식 확인

문서를 어떻게 chunking 할 것인지는 RAG 성능에 많은 영향을 미칩니다.

아래 예시에서는 단순히 PDF의 Page 단위에 맞춰서 문서를 chunking 하도록 했습니다.

In [ ]:
pdf_list = ['./data/sample2.pdf']

In [ ]:
# PDF에서 cid를 추출해서 ASCII 문자로 변환
def text_pruner(text, current_pdf_file):
    def replace_cid(match):
        ascii_num = int(match.group(1))
        try:
            return chr(ascii_num)
        except:
            return ''  # 변환 실패 시 빈 문자열로 처리
    cid_pattern = re.compile(r'\(cid:(\d+)\)')
    return re.sub(cid_pattern, replace_cid, text)

# PDF 파일 처리 함수
def process_pdf(pdf_file):
    print(f"Processing PDF file: {pdf_file}")
    docs = []
    source_name = pdf_file.split('/')[-1]
    type_name = source_name.split(' ')[-1].replace('.pdf', '')

    with pdfplumber.open(pdf_file) as pdf:
        # 페이지 단위 Chunking (단순화 된 방법)
        for page_number, page in enumerate(pdf.pages, start=1):
            page_text = page.extract_text()
            if page_text:
                pruned_text = text_pruner(page_text, pdf_file)
            else:
                pruned_text = ""
            # 텍스트 길이가 20 이상인 경우에만 Documnet로 저장
            if len(pruned_text) >= 20:  
                doc = Document(
                    page_content=pruned_text.replace('\n', ' '),
                    metadata={
                        "source": source_name,
                        "type": type_name,
                        "timestamp": datetime.now().isoformat()
                    }
                )
                docs.append(doc)
    if docs:
        load_document(docs)

def load_document(docs):
    # 동작방식을 확인하기 위해, 출력만 진행 (업데이트 예정)
    print("sample doc: ", docs[0])
    print("number of docs", len(docs))    

for pdf_file in pdf_list:
    process_pdf(pdf_file)


### OpenSearch를 지식 저장소로 활용

이제 지식 저장소로 OpenSearch를 활용하기 위해, 각 Document들을 보관합니다.

#### OpenSearch 클라이언트 생성

In [ ]:
from opensearchpy import OpenSearch, RequestsHttpConnection

In [ ]:
http_auth = (opensearch_user_id, opensearch_user_password)
os_client = OpenSearch(
            hosts=[
                {
                    'host': opensearch_domain_endpoint.replace("https://", ""),
                    'port': 443
                }
            ],
            http_auth=http_auth, 
            use_ssl=True,
            verify_certs=True,
            timeout=300,
            connection_class=RequestsHttpConnection
        )

#### OpenSearch에 `sample_pdf` 인덱스 생성

In [ ]:
with open('index_template.json', 'r') as f:
    index_body = json.load(f)

index_name = "sample2_pdf"
exists = os_client.indices.exists(index_name)

if exists:
    os_client.indices.delete(index=index_name)
    print("Existing index has been deleted. Create new one.")
else:
    print("Index does not exist, Create one.")

os_client.indices.create(index_name, body=index_body)

In [ ]:
from botocore.config import Config

In [ ]:
retry_config = Config(
        region_name=region,
        retries={
            "max_attempts": 10,
            "mode": "standard",
        },
    )
boto3_bedrock = boto3.client("bedrock-runtime", region_name=region, config=retry_config)

#### 벡터 임베딩 및 OpenSearch 벡터 저장/검색을 위한 클래스 생성

검색증강생성(RAG)의 자연어 기반 챗봇이 가능한 핵심 원리 중 하나는 벡터임베딩을 활용한 텍스트의 저장 및 검색입니다.

1. 자연어 텍스트를 벡터임베딩으로 변환해주는 Bedrock Embedding 모델을 정의하고,
2. OpenSearch에서 제공하는 벡터 저장/검색을 위한 클래스(`OpenSearchVectorSearch`)를 생성합니다.

In [ ]:
from langchain.embeddings import BedrockEmbeddings
from langchain_community.vectorstores import OpenSearchVectorSearch

In [ ]:
llmemb = BedrockEmbeddings(
    client=boto3_bedrock,
    model_id="amazon.titan-embed-g1-text-02"
)
dimension = 1536

vector_db = OpenSearchVectorSearch(
    index_name=index_name,
    opensearch_url=opensearch_domain_endpoint,
    embedding_function=llmemb,
    http_auth=http_auth,
)

#### 이제 문서 로드 함수 `load_document()`에 add_documents() 호출 구문을 추가하여, 실제 문서가 벡터화(vectorization)되어 OpenSearch에 추가되도록 합니다.

앞에서는 동작방식을 확인하기 위해, 문서를 print로만 출력하고 종료했었습니다.

In [ ]:
def load_document(docs):
    short_docs = docs[:1]
    vector_db.add_documents(short_docs)

for pdf_file in pdf_list:
    process_pdf(pdf_file)

실행이 끝난 후에 OpenSearch에서 `sample_pdf` 인덱스를 조회해보면, 텍스트와 이 텍스트의 문맥적 의미를 담는 벡터가 함께 저장된 것이 확인됩니다.

(아래 내용은 참고만 하셔도 됩니다)

<img src="image/uploader-1.png">

### 파일 업로더를 Streamlit 애플리케이션에 통합

이제 Streamlit에서 업로드 된 파일이 자동으로 OpenSearch의 벡터 저장소에 보관되도록 처리하겠습니다.

In [ ]:
%%writefile ../uploader.py
import os
import boto3
import re
import json
import pdfplumber
from RAGchatbot.utils.ssm import parameter_store
from datetime import datetime
from opensearchpy import OpenSearch, RequestsHttpConnection
from botocore.config import Config
from langchain_core.documents import Document
from langchain_community.embeddings import BedrockEmbeddings
from langchain_community.vectorstores import OpenSearchVectorSearch

index_name = 'sample_pdf'
region_name = 'us-west-2'
pm = parameter_store(region_name)
opensearch_user_id = pm.get_params(key="opensearch_user_id", enc=False)
opensearch_user_password = pm.get_params(key="opensearch_user_password", enc=True)
opensearch_domain_endpoint = pm.get_params(key="opensearch_domain_endpoint", enc=False)

def setup_opensearch_client():
    http_auth = (opensearch_user_id, opensearch_user_password)
    os_client = OpenSearch(
                hosts=[{'host': opensearch_domain_endpoint.replace("https://", ""), 'port': 443}],
                http_auth=http_auth, 
                use_ssl=True,
                verify_certs=True,
                connection_class=RequestsHttpConnection
            )
    return os_client
    
os_client = setup_opensearch_client()

retry_config = Config(
    region_name=region_name,
    retries={
        "max_attempts": 10,
        "mode": "standard",
    },
)
boto3_bedrock = boto3.client("bedrock-runtime", region_name=region_name, config=retry_config)

llmemb = BedrockEmbeddings(
    client=boto3_bedrock,
    model_id="amazon.titan-embed-g1-text-02"
)

http_auth = (opensearch_user_id, opensearch_user_password)
vector_db = OpenSearchVectorSearch(
    index_name=index_name,
    opensearch_url=opensearch_domain_endpoint,
    embedding_function=llmemb,
    http_auth=http_auth,
)

# PDF에서 cid를 추출해서 ASCII 문자로 변환하는 함수
def text_pruner(text, current_pdf_file):
    def replace_cid(match):
        ascii_num = int(match.group(1))
        try:
            return chr(ascii_num)
        except:
            return ''  # 변환 실패 시 빈 문자열로 처리
    cid_pattern = re.compile(r'\(cid:(\d+)\)')
    return re.sub(cid_pattern, replace_cid, text)

# PDF 파일 처리 함수
def process_pdf(pdf_file):
    docs = []
    source_name = pdf_file.name
    type_name = source_name.split(' ')[-1].replace('.pdf', '')

    with pdfplumber.open(pdf_file) as pdf:
        for page_number, page in enumerate(pdf.pages, start=1):
            page_text = page.extract_text()
            if page_text:
                pruned_text = text_pruner(page_text, pdf_file)
            else:
                pruned_text = ""
            if len(pruned_text) >= 20:  
                doc = Document(
                    page_content=pruned_text.replace('\n', ' '),
                    metadata={
                        "source": source_name,
                        "type": type_name,
                        "timestamp": datetime.now().isoformat()
                    }
                )
                docs.append(doc)
    if docs:
        load_document(docs)

def load_document(docs):
    vector_db.add_documents(docs)
    print("Done")


# 인덱스 생성 및 관리
def manage_index(os_client):
    with open('./RAGchatbot/index_template.json', 'r') as f:
        index_body = json.load(f)
        
    exists = os_client.indices.exists(index_name)
    if exists:
        os_client.indices.delete(index=index_name)
    else:
        os_client.indices.create(index_name, body=index_body)

def upload_and_process_file(uploaded_file):
    try:
        manage_index(os_client)
        process_pdf(uploaded_file)
        return True
    except Exception as e:
        print(f"Error processing file: {e}")
        return False

In [ ]:
%%writefile ../demo-app.py
from basic import get_conversation
import streamlit as st
from uploader import upload_and_process_file  # 추가된 부분

st.set_page_config(layout="wide")
st.title("Bedrock Q&A Chatbot")

model_options = [
    "anthropic.claude-instant-v1",
    "anthropic.claude-v2:1",
    "anthropic.claude-3-haiku-20240307-v1:0",
    "anthropic.claude-3-sonnet-20240229-v1:0"
]
st.sidebar.title("Model Selection (Claude)")
selected_model = st.sidebar.selectbox("모델 선택", model_options, index=3)

uploaded_file = st.file_uploader("파일을 업로드하세요", type=["pdf"])
prompt = st.text_input("프롬프트를 입력하세요.")
search_type = st.radio("Search Type", ["Basic", "Basic-RAG", "Hybrid-RAG"])

chat_box = st.empty()

if 'conversation' not in st.session_state or 'stream_handler' not in st.session_state:
    st.session_state.conversation, st.session_state.stream_handler = get_conversation(chat_box, selected_model)

def search_documents(search_type: str, prompt: str):
    if search_type == "Basic":
        st.session_state.stream_handler.reset_accumulated_text()
        st.session_state.conversation.predict(input=prompt)
    elif search_type == "Basic-RAG":
        st.write(f"문서 기본 검색: {prompt}")
    elif search_type == "Hybrid-RAG":
        st.write(f"하이브리드 검색: {prompt}")

if st.button("검색"):
    search_documents(search_type, prompt)

if uploaded_file is not None:
    res = upload_and_process_file(uploaded_file)
    if res:
        st.success("파일이 성공적으로 처리됐습니다.")
    else:
        st.error("파일 처리 중 오류가 발생했습니다.")

#### 이제 Streamlit 애플리케이션에서 PDF 파일을 업로드 했을 때 아래와 같이 표시됩니다

<img src="./image/uploader-2.png" width="800">